In [1]:
import numpy as np
import torch
from torchinfo import summary

import modules.dataloaders as dataloaders
import modules.models.model_CNV_imagenet as cnv_model
# import modules.model_CNV_AIMET_imagenet as cnv_aimet_model

import modules.models.model_CNV_AIMET_NoPadding_FPGA_imagenet as cnv_aimet_model

import modules.metrics as metrics
import modules.val_final_model as val_final_model
import modules.utils as utils

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [2]:
# from brevitas.export import export_onnx_qcdq
from brevitas.export import export_qonnx

# Load Model

In [3]:
AIMET = "YES"
if AIMET == "NO":
    model = cnv_model.CNV().to('cuda')
elif AIMET == "YES":
    model = cnv_aimet_model.CNV_AIMET().to('cuda')
else:
    print("Wrong Model")

In [4]:
NO_PADDING = True

### Torchinfo

In [5]:
if NO_PADDING:
    print("Using No Padding Model -> DIMS (230, 230)")
    print(summary(model, input_size=(1, 3, 230, 230)))
else:
    print("Using Padding Model -> DIMS (224, 224)")
    print(summary(model, input_size=(1, 3, 224, 224)))

Using No Padding Model -> DIMS (230, 230)


/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


Layer (type:depth-idx)                                                      Output Shape              Param #
CNV_AIMET                                                                   [1, 2]                    --
├─ModuleList: 1-107                                                         --                        (recursive)
│    └─QuantIdentity: 2-1                                                   [1, 3, 230, 230]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                  [1, 3, 230, 230]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                  [1, 3, 230, 230]          1
├─ModuleList: 1-108                                                         --                        (recursive)
│    └─QuantLinear: 2-163                                                   --                        (recursive)
│    │    └─WeightQuantProxyFromInjector: 3-202                             --                        (recursive)
├─ModuleList: 1

# Load Weights

In [6]:
# weights_folder = './experiments/test_v04__imagenetQuant__w2a4_PerChannel_FixedPoint/weights'
# model_ckpnt = weights_folder + '/BED_classifier__best_mean_F1.pt'

# weights_folder = './experiments/test_v10__AIMET/weights'
# model_ckpnt = weights_folder + '/BED_classifier__best_mean_F1.pt'

weights_folder = './experiments/test_v13__AIMET_BN_ReLU_added__NoPadding__QuantIdentity/weights'
model_ckpnt = weights_folder + '/BED_classifier__best_mean_F1.pt'

In [7]:
utils.load_checkpoint(
    model_path = model_ckpnt, 
    model = model,
    device = 'cuda'
)

Loading Model. Trained during 78 epochs


78

In [8]:
model.to('cpu')
model.eval();

# Test with DFire MINI Dataset: Train and Test

In [9]:
# import importlib
# importlib.reload(config)
# importlib.reload(dataloaders)

In [10]:
# train_dfire_mini_loader = dataloaders.get_dfire_mini_train_loader()
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Test DFire MINI

In [11]:
with torch.no_grad():
    val_metrics = val_final_model.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,                         
        device='cpu')

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 38.76it/s]

SMOKE -> Precision: 0.9500 - Recall: 0.8636 - Accuracy: 0.8667 - F1: 0.9048
FIRE -> Precision: 1.0000 - Recall: 1.0000 - Accuracy: 1.0000 - F1: 1.0000


In [12]:
print('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
print(val_metrics)


Testing with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT
{'Accuracy': [0.8666666746139526, 1.0], 'Precision': [0.949999988079071, 1.0], 'Recall': [0.8636363744735718, 1.0], 'F1': [0.9047619104385376, 1.0]}


# Test with DFire Full Test

In [13]:
test_dfire_loader = dataloaders.get_dfire_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306


In [27]:
with torch.no_grad():
    val_metrics = val_final_model.eval_fn(
        loader=test_dfire_loader, 
        model=model,                         
        device='cpu')

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [01:55<00:00,  1.72s/it]

SMOKE -> Precision: 0.9118 - Recall: 0.8619 - Accuracy: 0.8934 - F1: 0.8861
FIRE -> Precision: 0.9518 - Recall: 0.9063 - Accuracy: 0.9639 - F1: 0.9285


# Convert the Model to BIPOLAR OUT

In [15]:
# import brevitas.nn as qnn
# import torch.nn as nn

In [16]:
# class CNV_BIPOLAR_OUT(nn.Module):
#     def __init__(self, base_model):
#         super(CNV_BIPOLAR_OUT, self).__init__()
#         self.base_model = base_model
#         self.qnt_output = qnn.QuantIdentity(
#             quant_type='binary', 
#             scaling_impl_type='const',
#             bit_width=1, min_val=-1.0, max_val=1.0)

#     def forward(self, x):
#         x = self.base_model(x)
#         x = self.qnt_output(x)
#         return x

In [17]:
# cnv_bipolar_out = CNV_BIPOLAR_OUT(model).to('cpu')

### New Evaluation for BIPOLAR Out Model

In [18]:
# from tqdm import tqdm

# def eval_bipolar_fn(loader, model, device):
    
#     model.eval()
#     loop = tqdm(loader, desc='Validating', leave=True)

#     for batch_idx, (x, y) in enumerate(loop):
#         x, y = x.to(device), y.to(device)
#         yhat = model(x)

#         # print(y.shape)
#         # print(yhat.shape)
        
#         #yhat[yhat < 1] = 0
#         #yhat = torch.sigmoid(yhat.detach())
#         # print(f'Label {y} - Pred {yhat}')
#         metrics.precision_metric_cpu.update(yhat, y)
#         metrics.recall_metric_cpu.update(yhat, y)
#         metrics.accuracy_metric_cpu.update(yhat, y)
#         metrics.f1_metric_cpu.update(yhat, y)
    
#     precision = metrics.precision_metric_cpu.compute()
#     recall = metrics.recall_metric_cpu.compute()
#     accuracy = metrics.accuracy_metric_cpu.compute()
#     f1 = metrics.f1_metric_cpu.compute()
    
#     metrics.precision_metric_cpu.reset()
#     metrics.recall_metric_cpu.reset()
#     metrics.accuracy_metric_cpu.reset()
#     metrics.f1_metric_cpu.reset()

#     print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
#     print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
#     return (
#         {
#         'Accuracy': [accuracy[0].item(), accuracy[1].item()],
#         'Precision': [precision[0].item(), precision[1].item()],
#         'Recall': [recall[0].item(), recall[1].item()],
#         'F1': [f1[0].item(), f1[1].item()] 
#         }
#     )

### Mini Test

In [19]:
# with torch.no_grad():
#     val_metrics = eval_bipolar_fn(
#         loader=test_dfire_mini_loader, 
#         model=cnv_bipolar_out,                         
#         device='cpu')

# Export Bipolar to QONNX

In [20]:
# onnx_no_bipolar_filename = 'Best_F1_AIMET__NO_Bipolar.onnx'
# export_qonnx(cnv_bipolar_out, torch.randn(1, 3, 224, 224), onnx_no_bipolar_filename);

In [21]:
# onnx_bipolar_filename = 'Best_F1_AIMET__Bipolar.onnx'
# export_qonnx(cnv_bipolar_out, torch.randn(1, 3, 224, 224), onnx_bipolar_filename);

# Evaluate QONNX Model

QONNX exec does not work. FINN exec must be used inside FINN docker.

In [22]:
# from qonnx.util.cleanup import cleanup as qonnx_cleanup
# from qonnx.core.modelwrapper import ModelWrapper
# import qonnx.core.onnx_exec as qonnx_exec

# from qonnx.transformation.infer_shapes import InferShapes
# from qonnx.transformation.fold_constants import FoldConstants
# from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

### Clean before evaluation

In [23]:
# clean_no_bipolar_onnx_filename = 'Best_F1_AIMET__NO_Bipolar__CLEAN.onnx'
# qonnx_cleanup(onnx_no_bipolar_filename, out_file=clean_no_bipolar_onnx_filename)

# clean_bipolar_onnx_filename = 'Best_F1_AIMET__Bipolar__CLEAN.onnx'
# qonnx_cleanup(onnx_bipolar_filename, out_file=clean_bipolar_onnx_filename)

In [24]:
# def eval_qonnx(loader, model_filename):
    
#     model_wrapped = ModelWrapper(model_filename)
#     # model_wrapped = model_wrapped.transform(InferShapes())
#     # model_wrapped = model_wrapped.transform(FoldConstants())
#     # model_wrapped = model_wrapped.transform(GiveUniqueNodeNames())
#     # model_wrapped = model_wrapped.transform(GiveReadableTensorNames())
#     # model_wrapped = model_wrapped.transform(RemoveStaticGraphInputs())
    
#     loop = tqdm(loader, desc='Validating', leave=True)

#     for batch_idx, (x, y) in enumerate(loop):
#         x, y = x.to('cpu'), y.to('cpu')
#         img = x.detach().numpy()
#         inp_dict = {"onnx::Mul_0": img}
#         yhat = qonnx_exec.execute_onnx(model_wrapped, inp_dict)
#         # print(y.shape)
#         # print(yhat.shape)

#         yhat = torch.tensor(yhat["global_out"], dtype=torch.float32)
#         # print(f'Label {y} - Pred {yhat}')

#         # yhat[yhat < 1] = 0
#         yhat = torch.sigmoid(yhat.detach())

#         print(f'Label {y} - Pred {yhat}')
#         metrics.precision_metric_cpu.update(yhat, y)
#         metrics.recall_metric_cpu.update(yhat, y)
#         metrics.accuracy_metric_cpu.update(yhat, y)
#         metrics.f1_metric_cpu.update(yhat, y)
    
#     precision = metrics.precision_metric_cpu.compute()
#     recall = metrics.recall_metric_cpu.compute()
#     accuracy = metrics.accuracy_metric_cpu.compute()
#     f1 = metrics.f1_metric_cpu.compute()
    
#     metrics.precision_metric_cpu.reset()
#     metrics.recall_metric_cpu.reset()
#     metrics.accuracy_metric_cpu.reset()
#     metrics.f1_metric_cpu.reset()

#     print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
#     print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
#     return (
#         {
#         'Accuracy': [accuracy[0].item(), accuracy[1].item()],
#         'Precision': [precision[0].item(), precision[1].item()],
#         'Recall': [recall[0].item(), recall[1].item()],
#         'F1': [f1[0].item(), f1[1].item()] 
#         }
#     )

In [25]:
# val_metrics = eval_qonnx(
#     loader=test_dfire_mini_loader, 
#     model_filename=clean_no_bipolar_onnx_filename)

In [26]:
# val_metrics = eval_qonnx(
#     loader=test_dfire_mini_loader, 
#     model_filename=clean_bipolar_onnx_filename)